In [2]:
import pandas as pd
import xmlrpc.client

In [3]:
import xmlrpc.client
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'account.invoice.report',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

,sortable,string,searchable,required,manual,readonly,states,depends,groups,company_dependent,type,store,relation,digits,translate,size,selection,domain,help
date_due,True,Due Date,True,False,False,True,{},[],False,False,date,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
partner_bank_id,True,Bank Account,True,False,False,True,{},[],False,False,many2one,True,res.partner.bank,NaN,NaN,NaN,NaN,NaN,NaN
payment_term,True,Payment Term,True,False,False,True,{},[],False,False,many2one,True,account.payment.term,NaN,NaN,NaN,NaN,NaN,NaN
nbr,True,# of Invoices,True,False,False,True,{},[],False,False,integer,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
journal_id,True,Journal,True,False,False,True,{},[],False,False,many2one,True,account.journal,NaN,NaN,NaN,NaN,NaN,NaN
currency_id,True,Currency,True,False,False,True,{},[],False,False,many2one,True,res.currency,NaN,NaN,NaN,NaN,NaN,NaN
product_qty,True,Product Quantity,True,False,False,True,{},[],False,False,float,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_currency_price_average,False,Average Price,False,False,False,True,{},[],False,False,float,False,NaN,"[16, 2]",NaN,NaN,NaN,NaN,NaN
id,True,Id,True,False,False,True,{},[],False,False,integer,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
account_id,True,Account,True,False,False,True,{},[],False,False,many2one,True,account.account,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import xmlrpc.client
import pandas as pd
from datetime import datetime
import os

# ---------------- Conexión ----------------
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ---------------- Campos a extraer ----------------
campos = [
    'product_marca',
    'product_cost',
    'price_total',
    'date'
]

# ---------------- Carpeta de salida ----------------
output_dir = "/home/donsson/proyectos/SIMON/exports"
os.makedirs(output_dir, exist_ok=True)

# ---------------- Rango de años ----------------
start_year = 2020
end_year   = datetime.now().year   # hasta hoy
batch_size = 20000   # bloques grandes = menos llamadas

for year in range(start_year, end_year + 1):
    print(f"\n📌 Extrayendo registros del año {year}...")

    # Dominio filtrando solo el año
    domain = [
        ['date', '>=', f"{year}-01-01"],
        ['date', '<=', f"{year}-12-31"],
        ['state', '=', 'paid']
    ]

    # Saber cuántos registros hay en ese año
    total = models.execute_kw(
        db, uid, password,
        'account.invoice.report', 'search_count',
        [domain]
    )
    print(f"   Total de registros en {year}: {total}")

    if total == 0:
        print(f"⚠️ No había registros en {year}")
        continue

    # Acumulador en lista (más rápido que concatenar en cada ciclo)
    registros_list = []
    offset = 0

    while offset < total:
        print(f"   Descargando lote desde offset {offset}...")
        registros = models.execute_kw(
            db, uid, password,
            'account.invoice.report', 'search_read',
            [domain],
            {'fields': campos, 'limit': batch_size, 'offset': offset}
        )
        registros_list.extend(registros)  # agregamos a la lista
        offset += batch_size

    # Convertir a DataFrame una sola vez
    df_year = pd.DataFrame(registros_list)

    # Guardar Excel
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(output_dir, f"facturas_pagadas_{year}_{timestamp}.xlsx")
    df_year.to_excel(filename, index=False)
    print(f"✅ Archivo generado: {filename} ({len(df_year)} registros)")




📌 Extrayendo registros del año 2020...
   Total de registros en 2020: 124820
   Descargando lote desde offset 0...
   Descargando lote desde offset 20000...
   Descargando lote desde offset 40000...
   Descargando lote desde offset 60000...
   Descargando lote desde offset 80000...
   Descargando lote desde offset 100000...
   Descargando lote desde offset 120000...
✅ Archivo generado: /home/donsson/proyectos/SIMON/exports/facturas_pagadas_2020_20250923_101959.xlsx (124820 registros)

📌 Extrayendo registros del año 2021...
   Total de registros en 2021: 153867
   Descargando lote desde offset 0...
   Descargando lote desde offset 20000...
   Descargando lote desde offset 40000...
   Descargando lote desde offset 60000...
   Descargando lote desde offset 80000...
   Descargando lote desde offset 100000...
   Descargando lote desde offset 120000...
   Descargando lote desde offset 140000...
✅ Archivo generado: /home/donsson/proyectos/SIMON/exports/facturas_pagadas_2021_20250923_110519.x

In [14]:
df_total.head(30)

,date,price_total,product_cost,id,product_marca
0,2025-09-22,60200.0,21847.84,1619355,DONSSON
1,2025-09-22,70200.0,46625.18,1619231,BALDWIN
2,2025-09-22,82100.0,55892.34,1618763,BALDWIN
3,2025-09-22,54600.0,37345.78,1618660,BALDWIN
4,2025-09-22,70700.0,24977.50,1618975,DONSSON
5,2025-09-22,54400.0,19836.00,1619138,AUT*PARTS
6,2025-09-22,61600.0,41400.60,1619322,BALDWIN
7,2025-09-22,53800.0,13845.12,1619196,DONSSON
8,2025-09-22,175800.0,80492.78,1618621,DONSSON
9,2025-09-22,83200.0,51875.42,1619407,BALDWIN
